In [5]:
import pandas as pd
import ast
from pyspark.sql import SparkSession

In [15]:
from pyspark.sql.functions import col, regexp_extract,round, sqrt, pow, lit

In [6]:
spark = SparkSession.builder.appName('export_sff').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/21 01:43:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
events = spark.read.csv('Data/events.csv',header=True,inferSchema=True,sep=';')

In [36]:
events_df = events.filter(events.type=='Shot')

In [37]:
def split_location(df):
    df_l = df.withColumn("shot_location_x", regexp_extract(col("location"), r'\[(.*?),', 1).cast("float")) \
             .withColumn("shot_location_y", regexp_extract(col("location"), r', (.*?)\]', 1).cast("float")).drop('location')
    return df_l

In [38]:
events_df = split_location(events_df)

In [ ]:
import math
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

# Define the goalpost coordinates
goal_x1, goal_y1 = 120, 36  # Left post
goal_x2, goal_y2 = 120, 44  # Right post

# Function to calculate the angle
def calculate_shot_angle(shot_x, shot_y):
    # Vectors to the posts
    u_x, u_y = goal_x1 - shot_x, goal_y1 - shot_y
    v_x, v_y = goal_x2 - shot_x, goal_y2 - shot_y
    
    # Dot product and magnitudes
    dot_product = u_x * v_x + u_y * v_y
    magnitude_u = math.sqrt(u_x**2 + u_y**2)
    magnitude_v = math.sqrt(v_x**2 + v_y**2)
    
    # Avoid division by zero
    if magnitude_u == 0 or magnitude_v == 0:
        return 0.0
    
    # Calculate angle in radians
    angle_radians = math.acos(dot_product / (magnitude_u * magnitude_v))
    
    # Convert to degrees if needed (optional)
    angle_degrees = math.degrees(angle_radians)
    return angle_degrees

# Register the function as a UDF
calculate_shot_angle_udf = udf(calculate_shot_angle, FloatType())

# Apply the UDF to your PySpark DataFrame
# Assuming 'location_x' and 'location_y' are the column names for shot coordinates
events_df = events_df.withColumn(
    "shot_angle",
    calculate_shot_angle_udf(events_df["shot_location_x"], events_df["shot_location_y"])
)

# Show the result
events_df.select("shot_location_x", "shot_location_y", "shot_angle").show()


+---------------+---------------+----------+
|shot_location_x|shot_location_y|shot_angle|
+---------------+---------------+----------+
|          105.6|           44.0| 29.054602|
|           98.6|           25.2| 14.520617|
|          105.5|           47.3| 25.108313|
|          113.4|           38.7| 61.014557|
|          106.2|           36.8| 30.870586|
|          108.3|           39.0| 37.520866|
|           85.6|           34.4|  12.92981|
|          112.5|           42.5| 52.224316|
|          106.1|           32.5| 25.469059|
|           83.5|           48.0| 11.945132|
|          108.1|           39.3|  37.05121|
|          105.5|           57.6| 12.961155|
|          110.9|           25.6| 14.870527|
|           98.9|           24.9| 14.404465|
|          114.8|           44.1|  56.19886|
|          107.2|           35.0| 30.644846|
|          107.5|           24.9| 15.192171|
|           88.5|           38.1| 14.422748|
|          109.6|           41.5| 41.388515|
|         

In [33]:
events_df.filter(events_df.shot_angle>90).orderBy('shot_angle',ascending=False).show(10)

+-----+--------------------+------------------------------+------------------+--------------------+------------+--------------+---------------+---------------+------------+---------+--------+------------------------+-------------------+------------------+------------------+--------------------+-----------------------+------------------+-------------------+--------------------+---------------+--------------------+-----+--------------------+--------+------+---------------+----------+---------------------+-------------+--------------+----------+-------------+--------------+-----------------+----------------+-----------+-----------+------------+--------------+-----------------+----------------+-----------+---------+------+--------------+--------------------+---------+--------------------+----------+---------------+------------------+--------------------+------+---------------+--------------+------------------+---------------+--------------------+----------------+---------------+-----------

In [34]:
matches = spark.read.csv('Data/matches.csv',header=True,inferSchema=True)

In [35]:
matches.filter(matches.match_id==266254).show()

+--------+----------+-------------------+---------------+---------+---------+----------+----------+----------+----------+-----------------+----------------+-------+--------------------+--------------------+
|match_id|match_date|           kick_off|    competition|   season|home_team| away_team|home_score|away_score|match_week|competition_stage|         stadium|referee|       home_managers|       away_managers|
+--------+----------+-------------------+---------------+---------+---------+----------+----------+----------+----------+-----------------+----------------+-------+--------------------+--------------------+
|  266254|2016-02-14|2024-12-21 21:30:00|Spain - La Liga|2015/2016|Barcelona|Celta Vigo|         6|         1|        24|   Regular Season|Spotify Camp Nou|   NULL|Luis Enrique Mart...|Manuel Eduardo Be...|
+--------+----------+-------------------+---------------+---------+---------+----------+----------+----------+----------+-----------------+----------------+-------+--------

In [39]:
import math
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

# Define the goalpost coordinates
goal_x1, goal_y1 = 120, 36  # Left post
goal_x2, goal_y2 = 120, 44  # Right post

# Function to calculate the angle
def calculate_shot_angle(shot_x, shot_y):
    # Vectors to the posts
    u_x, u_y = goal_x1 - shot_x, goal_y1 - shot_y
    v_x, v_y = goal_x2 - shot_x, goal_y2 - shot_y
    
    # Dot product and magnitudes
    dot_product = u_x * v_x + u_y * v_y
    magnitude_u = math.sqrt(u_x**2 + u_y**2)
    magnitude_v = math.sqrt(v_x**2 + v_y**2)
    
    # Avoid division by zero
    if magnitude_u == 0 or magnitude_v == 0:
        return 0.0
    
    # Calculate angle in radians
    angle_radians = math.acos(dot_product / (magnitude_u * magnitude_v))
    
    # Convert to degrees
    return math.degrees(angle_radians)

# Define a function to add the shot angle column to the DataFrame
def get_shot_angle(df):
    # Register the function as a UDF
    calculate_shot_angle_udf = udf(calculate_shot_angle, FloatType())
    
    # Apply the UDF to the DataFrame
    df = df.withColumn(
        "shot_angle",
        calculate_shot_angle_udf(df["shot_location_x"], df["shot_location_y"])
    )
    return df

# Use the function
events_df = get_shot_angle(events_df)

# Show the result
events_df.select("shot_location_x", "shot_location_y", "shot_angle").show()

+---------------+---------------+----------+
|shot_location_x|shot_location_y|shot_angle|
+---------------+---------------+----------+
|          105.6|           44.0| 29.054602|
|           98.6|           25.2| 14.520617|
|          105.5|           47.3| 25.108313|
|          113.4|           38.7| 61.014557|
|          106.2|           36.8| 30.870586|
|          108.3|           39.0| 37.520866|
|           85.6|           34.4|  12.92981|
|          112.5|           42.5| 52.224316|
|          106.1|           32.5| 25.469059|
|           83.5|           48.0| 11.945132|
|          108.1|           39.3|  37.05121|
|          105.5|           57.6| 12.961155|
|          110.9|           25.6| 14.870527|
|           98.9|           24.9| 14.404465|
|          114.8|           44.1|  56.19886|
|          107.2|           35.0| 30.644846|
|          107.5|           24.9| 15.192171|
|           88.5|           38.1| 14.422748|
|          109.6|           41.5| 41.388515|
|         